In [1]:
import yfinance as yf
import pandas as pd

In [2]:
def get_info(ticker: str) -> dict:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    return {
        'stock info': {
            'ticker': ticker,
            'industry': info.get('industry', 'n/a'),
            'sector': info.get('sector', 'n/a'),
            'current price': info.get('currentPrice', None),
            'analyst low price': info.get('targetLowPrice', None),
            'analyst median price': info.get('targetMedianPrice', None),
            'analyst high price': info.get('targetHighPrice', None),
        },
        
        'metrics': {
            'sector': info.get('sector', 'n/a'),
            
            # risk metrics
            'beta': info.get('beta', 1), # market risk (lower = less risky)
            'risk rating': info.get('overallRisk', None), # risk rating out of 10 (lower = better)
            
            # valuation metrics
            'forward P/E': info.get('forwardPE', None), # projected price to earnings (lower = better)
            'PEG ratio': info.get('trailingPegRatio', None), # price to earnings adjusted for growth (lower = better, ideally < 1)
            'price to book P/B': info.get('priceToBook', None), # price to book ratio (lower = better)
            'enterprise to ebitda': info.get('enterpriseToEbitda', None), # enterprise value to ebitda (lower = better)
            
            # growth metrics
            'revenue growth': info.get('revenueGrowth', None), # percent growth in revenue (higher = better)
            'earnings growth': info.get('earningsGrowth', None), # percent growth in earnings (higher = better)
            
            # profitability metrics
            'profit margin': info.get('profitMargins', None), # percent of revenue that is profit (higher = better)
            'return on equity': info.get('returnOnEquity', None), # percent return on equity (higher = better)
            
            # financial health metrics
            'cash per share': info.get('totalCashPerShare', None), # cash per share (higher = better)
            'short ratio': info.get('shortRatio', None), # number of days to cover short positions (lower = better)
            
            # analyst expectations. take it with a grain of salt
            'analyst median growth': round(100*(info.get('targetMedianPrice', 0)/info.get('currentPrice', 1) - 1), 2),
            'recommendation mean': info.get('recommendationMean', None), # 1 to 5, with 1 being a strong buy and 5 being a strong sell
        }
    }

In [3]:
def calculate_market_sector_average(tickers: str, sector:str) -> dict:
    data = [get_info(ticker) for ticker in tickers]
    df = pd.DataFrame([d['metrics'] for d in data])
    
    average = {}
    sector_df = df[df['sector'] == sector].copy()
    sector_df.drop(columns='sector', inplace=True)
    df.drop(columns='sector', inplace=True)

    average['Market'] = df.mean()
    average['Sector'] = sector_df.mean()

    return pd.DataFrame(average)

In [4]:
def compare_stock(ticker: str) -> dict:
    # market tickers
    snp_tickers = pd.read_csv('../blogs/data/snp500.csv')['ticker'].tolist()
    
    info = get_info(ticker)
    market_info = calculate_market_sector_average(snp_tickers, info["stock info"]["sector"])
    stock_metrics = info['metrics']
    
    market_info[ticker] = stock_metrics
    market_info['Sector Diff'] = (market_info[ticker] - market_info['Sector']) / market_info['Sector']
    market_info['Market Diff'] = (market_info[ticker] - market_info['Market']) / market_info['Market']
    market_info = market_info[[ticker, 'Sector', 'Sector Diff', 'Market', 'Market Diff']]
    
    print(f"{ticker}")
    print(f'Sector: {info["stock info"]["sector"]}')
    print(f'current price: {info["stock info"]["current price"]}')
    print(f'Analyst projections: low {info["stock info"]["analyst low price"]}, median {info["stock info"]["analyst median price"]}, high {info["stock info"]["analyst high price"]}')
    print('\n Risk Metrics (lower less risk):')
    print(market_info.loc[['beta', 'risk rating']])
    print('\n Valuation Metrics (lower better):')
    print(market_info.loc[['forward P/E', 'PEG ratio', 'price to book P/B', 'enterprise to ebitda']])
    print('\n Growth Metrics (higher better):')
    print(market_info.loc[['revenue growth', 'earnings growth']])
    print('\n Profitability Metrics (higher better):')
    print(market_info.loc[['profit margin', 'return on equity']])
    print('\n Financial Health Metrics:')
    print(market_info.loc[['cash per share', 'short ratio']])
    print('\n Analyst Expectations:')
    print(market_info.loc[['analyst median growth', 'recommendation mean']])

In [5]:
compare_stock('NVDA')

NVDA
Sector: Technology
current price: 107.27
Analyst projections: low 50.3, median 140.0, high 200.0

 Risk Metrics (lower less risk):
             NVDA    Sector Sector Diff    Market Market Diff
beta         1.68  1.179603    0.424209  1.056173    0.590648
risk rating     8  5.166667    0.548387  5.522088    0.448727

 Valuation Metrics (lower better):
                           NVDA     Sector Sector Diff     Market Market Diff
forward P/E           28.681818  22.817588    0.257005  20.252710    0.416197
PEG ratio                1.2185   1.911941    -0.36269   2.791483   -0.563494
price to book P/B     53.688686  12.842985     3.18039   8.701044    5.170373
enterprise to ebitda     53.143  31.542263    0.684819  19.085962    1.784403

 Growth Metrics (higher better):
                  NVDA    Sector Sector Diff    Market Market Diff
revenue growth   2.621  0.111909   22.420796  0.056054   45.758393
earnings growth    6.5  1.687729     2.85133  1.202233    4.406607

 Profitability M

In [6]:
compare_stock('CRWD')

CRWD
Sector: Technology
current price: 217.89
Analyst projections: low 253.0, median 350.0, high 540.0

 Risk Metrics (lower less risk):
              CRWD    Sector Sector Diff    Market Market Diff
beta         1.095  1.179603   -0.071721  1.056173    0.036762
risk rating     10  5.166667    0.935484  5.522088    0.810909

 Valuation Metrics (lower better):
                           CRWD     Sector Sector Diff     Market Market Diff
forward P/E           45.018593  22.817588    0.972978  20.252710    1.222843
PEG ratio                0.9247   1.911941   -0.516355   2.791483   -0.668742
price to book P/B     20.898716  12.842985    0.627248   8.701044    1.401863
enterprise to ebitda    345.497  31.542263    9.953463  19.085962   17.102153

 Growth Metrics (higher better):
                   CRWD    Sector Sector Diff    Market Market Diff
revenue growth     0.33  0.111909    1.948822  0.056054    4.887169
earnings growth  82.333  1.687729   47.783318  1.202233   67.483408

 Profitab

In [7]:
compare_stock('INTC')

INTC
Sector: Technology
current price: 21.48
Analyst projections: low 17.0, median 35.0, high 68.0

 Risk Metrics (lower less risk):
              INTC    Sector Sector Diff    Market Market Diff
beta         1.047  1.179603   -0.112413  1.056173   -0.008685
risk rating      4  5.166667   -0.225806  5.522088   -0.275636

 Valuation Metrics (lower better):
                           INTC     Sector Sector Diff     Market Market Diff
forward P/E           11.246074  22.817588   -0.507131  20.252710   -0.444713
PEG ratio                 0.391   1.911941   -0.795496   2.791483   -0.859931
price to book P/B      0.797091  12.842985   -0.937936   8.701044   -0.908391
enterprise to ebitda     11.409  31.542263   -0.638295  19.085962   -0.402231

 Growth Metrics (higher better):
                  INTC    Sector Sector Diff    Market Market Diff
revenue growth  -0.009  0.111909   -1.080422  0.056054   -1.160559
earnings growth   None  1.687729         NaN  1.202233         NaN

 Profitability M